In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [5]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=['Unnamed: 0','fechapostulacion'])

In [4]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         uint32
idpostulante    object
dtypes: object(1), uint32(1)
memory usage: 75.6+ MB


In [5]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'])

In [6]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['Unnamed: 0','nombre_numerico'])

In [7]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [8]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante       504407 non-null object
fechanacimiento    504407 non-null object
sexo               504407 non-null category
nombre             447909 non-null category
estado             447909 non-null category
dtypes: category(3), object(2)
memory usage: 13.0+ MB


In [9]:
postulantes_merge.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NaN,NaN
1,Nzz,nan,NO_DECLARA,NaN,NaN
2,ZX1,nan,NO_DECLARA,NaN,NaN
3,Nq5,nan,NO_DECLARA,NaN,NaN
4,ebE,1952-07-07,MASC,Posgrado,Graduado


In [8]:
#Voy a agregar una columna con la edad ya calculada(estimando por 2017-año nacimiento) para cada registro
postulantes_merge["año"] = postulantes_merge[postulantes_merge["fechanacimiento"] != 'nan']\
        ["fechanacimiento"].apply(lambda x: x[0:4])
postulantes_merge["edad"] = postulantes_merge[postulantes_merge["año"].notna()] \
        ["año"].apply(lambda x: 2017 - int(x))

In [9]:
#Elimino columna fechanacimiento y año que ya no me sirven
postulantes_merge = postulantes_merge.drop(columns=['fechanacimiento','año'])

In [10]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante    504407 non-null object
sexo            504407 non-null category
nombre          447909 non-null category
estado          447909 non-null category
edad            477918 non-null float64
dtypes: category(3), float64(1), object(1)
memory usage: 13.0+ MB


In [11]:
#Elimino NaNs
postulantes_merge = postulantes_merge[postulantes_merge['nombre'].notnull() &
                                     postulantes_merge['estado'].notnull() &
                                     postulantes_merge['edad'].notnull()]

In [14]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438357 entries, 4 to 504405
Data columns (total 5 columns):
idpostulante    438357 non-null object
sexo            438357 non-null category
nombre          438357 non-null category
estado          438357 non-null category
edad            438357 non-null float64
dtypes: category(3), float64(1), object(1)
memory usage: 11.3+ MB


In [12]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
postulantes_merge["estado_code"] = lb_make1.fit_transform(postulantes_merge["estado"])

lb_make2 = LabelEncoder()
postulantes_merge["sexo_code"] = lb_make2.fit_transform(postulantes_merge["sexo"])

lb_make3 = LabelEncoder()
postulantes_merge["nombre_code"] = lb_make3.fit_transform(postulantes_merge["nombre"])

postulantes_merge.drop(columns=['sexo','nombre','estado'],inplace=True)
postulantes_merge[['sexo_code','nombre_code','estado_code','edad']] = postulantes_merge[['sexo_code','nombre_code','estado_code','edad']].apply(pd.to_numeric,downcast='unsigned')

In [13]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438357 entries, 4 to 504405
Data columns (total 5 columns):
idpostulante    438357 non-null object
edad            438357 non-null uint16
estado_code     438357 non-null uint8
sexo_code       438357 non-null uint8
nombre_code     438357 non-null uint8
dtypes: object(1), uint16(1), uint8(3)
memory usage: 8.8+ MB


In [17]:
postulantes_merge.to_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/POSTULANTES_FINAL.csv')

# Agrego la informacion a postulaciones

In [6]:
merge_postulaciones = pd.merge(postulaciones,postulantes_merge,how='left',on='idpostulante')

In [7]:
#Creo columna con un 1 para indicar postulacion
merge_postulaciones['se_postulo'] = 1
merge_postulaciones['se_postulo'] = merge_postulaciones[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [8]:
merge_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 7 columns):
idaviso         uint32
idpostulante    object
edad            float64
estado_code     float64
sexo_code       float64
nombre_code     float64
se_postulo      uint8
dtypes: float64(4), object(1), uint32(1), uint8(1)
memory usage: 333.8+ MB


In [21]:
merge_postulaciones[merge_postulaciones['edad'].isnull()]

,idaviso,idpostulante,edad,estado_code,sexo_code,nombre_code,se_postulo
566,1112264812,1zKzwB,NaN,NaN,NaN,NaN,1
567,1111105892,1zKzwB,NaN,NaN,NaN,NaN,1
568,1112202363,1zKzwB,NaN,NaN,NaN,NaN,1
569,1112344620,1zKzwB,NaN,NaN,NaN,NaN,1
570,1112281337,NM1J0z,NaN,NaN,NaN,NaN,1
571,1112334640,NM1J0z,NaN,NaN,NaN,NaN,1
572,1112289378,NM1J0z,NaN,NaN,NaN,NaN,1
573,1112296073,NM1J0z,NaN,NaN,NaN,NaN,1
574,1112310231,NM1J0z,NaN,NaN,NaN,NaN,1
575,1112195462,NM1J0z,NaN,NaN,NaN,NaN,1


In [9]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
merge_sin_nan = merge_postulaciones[merge_postulaciones['nombre_code'].notnull() &
                                    merge_postulaciones['estado_code'].notnull() & 
                                    merge_postulaciones['edad'].notnull()]
merge_sin_nan = merge_sin_nan[merge_sin_nan['nombre_code'].notnull() &
                                    merge_sin_nan['estado_code'].notnull()]
merge_sin_nan[['edad','estado_code','sexo_code','nombre_code']] = merge_sin_nan[['edad','estado_code','sexo_code','nombre_code']].apply(pd.to_numeric,downcast='unsigned')
merge_sin_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6327947 entries, 0 to 6603751
Data columns (total 7 columns):
idaviso         uint32
idpostulante    object
edad            uint16
estado_code     uint8
sexo_code       uint8
nombre_code     uint8
se_postulo      uint8
dtypes: object(1), uint16(1), uint32(1), uint8(4)
memory usage: 156.9+ MB


# Agrego datos de vistas que serian los postulantes que ven algo y no se postulan

In [10]:
dtypes = pd.Series({'idAviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

vistas = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Vistas/Vistas-merge.csv',dtype=column_types,parse_dates=['timestamp'],
                     infer_datetime_format=True)
vistas = vistas.drop(columns=['Unnamed: 0'])
#Considero unica vista de aviso por postulante (no me importa ni horario ni cantidad solo que lo vea)
vistas.drop(columns=['timestamp'],inplace=True)
vistas.drop_duplicates(subset=['idAviso','idpostulante'],inplace=True)
vistas['se_postulo'] = 0
vistas['se_postulo'] = vistas[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [11]:
vistas.rename(index=str,columns={'idAviso':'idaviso'},inplace=True)
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 172.2+ MB


# Agrego datos de avisos y lo mergeo con las vistas

In [12]:
avisos = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/optimized_avisos_detalle.csv')
avisos.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [30]:
avisos.isnull().sum()

idaviso                      0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [13]:
#Hago un merge de avisos y vistas
vistas_y_avisos = pd.merge(vistas,avisos,how='inner',on='idaviso')

In [14]:
vistas_y_avisos[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']] = vistas_y_avisos[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']].apply(pd.to_numeric,downcast='unsigned')
vistas_y_avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5542237 entries, 0 to 5542236
Data columns (total 8 columns):
idaviso                      uint32
idpostulante                 object
se_postulo                   uint8
nombre_area_code             uint8
denominacion_empresa_code    uint16
nivel_laboral_code           uint8
tipo_de_trabajo_code         uint8
nombre_zona_code             uint8
dtypes: object(1), uint16(1), uint32(1), uint8(5)
memory usage: 142.7+ MB


# Merge de todos

In [ ]:
#Puede que haya vistas que enrealidad el postulante luego se postulo asi que verifico con el dataset merge
outer = pd.merge(merge_sin_nan,vistas_y_avisos,how='outer',on=['idaviso','idpostulante'])

In [51]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
outer = outer[outer['nombre_code'].notnull() &
            outer['estado_code'].notnull() & 
            outer['edad'].notnull() &
             outer['nombre_area_code'].notnull() &
             outer['denominacion_empresa_code'].notnull() &
             outer['nivel_laboral_code'].notnull() &
             outer['tipo_de_trabajo_code'].notnull() &
             outer['nombre_zona_code'].notnull()]

MemoryError: 

In [30]:
outer.se_postulo_x.value_counts()

1.0    1674375
Name: se_postulo_x, dtype: int64

In [31]:
#Dejo solo los que no aparecen ya en postulaciones
trainingSet = outer[(outer['se_postulo_x'].isnull() & outer['se_postulo_y'] == 0) &
                   (outer['se_postulo_x'] == 1 & outer['se_postulo_y'].isnull())]

In [32]:
trainingSet.drop('se_postulo_x',axis=1,inplace=True)
trainingSet.drop('Unnamed: 0',axis=1,inplace=True)
trainingSet.rename(index=str,columns={'se_postulo_y':'se_postulo'},inplace=True)

In [33]:
trainingSet.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [34]:
postMasVistas = pd.concat([trainingSet,merge_sin_nan])

In [37]:
postMasVistas.isnull().sum()

denominacion_empresa_code    6322780
edad                               0
estado_code                        0
idaviso                            0
idpostulante                       0
nivel_laboral_code           6322780
nombre_area_code             6322780
nombre_code                        0
nombre_zona_code             6322780
se_postulo                         0
sexo_code                          0
tipo_de_trabajo_code         6322780
dtype: int64

In [38]:
postMasVistas = postMasVistas[postMasVistas['denominacion_empresa_code'].notnull() &
                             postMasVistas['nivel_laboral_code'].notnull() &
                             postMasVistas['nombre_area_code'].notnull() &
                             postMasVistas['nombre_zona_code'].notnull() &
                             postMasVistas['tipo_de_trabajo_code'].notnull()]

In [39]:
postMasVistas.isnull().sum()

denominacion_empresa_code    0
edad                         0
estado_code                  0
idaviso                      0
idpostulante                 0
nivel_laboral_code           0
nombre_area_code             0
nombre_code                  0
nombre_zona_code             0
se_postulo                   0
sexo_code                    0
tipo_de_trabajo_code         0
dtype: int64

In [40]:
postMasVistas.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [42]:
#postMasVistas.to_csv('/home/fabrizio/env/NaventDatosTP/Data/postulaciones_mas_vistas_mas_avisos.csv')

In [92]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistas.drop('idpostulante',axis=1)

In [95]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistas.drop('idaviso',axis=1)

In [97]:
postMasVistasCopiaSinPostulanteParaCorr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1674375 entries, 23 to 6322779
Data columns (total 10 columns):
denominacion_empresa_code    1674375 non-null float64
edad                         1674375 non-null float64
estado_code                  1674375 non-null float64
nivel_laboral_code           1674375 non-null float64
nombre_area_code             1674375 non-null float64
nombre_code                  1674375 non-null float64
nombre_zona_code             1674375 non-null float64
se_postulo                   1674375 non-null float64
sexo_code                    1674375 non-null float64
tipo_de_trabajo_code         1674375 non-null float64
dtypes: float64(10)
memory usage: 140.5+ MB


In [104]:
postMasVistasCopiaSinPostulanteParaCorr.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [ ]:
#Dropeo las columnas que use para codificar
postulantes_merge = postulantes_merge.drop(columns=['sexo','nombre','estado'])
postulantes_merge.info()

In [ ]:
postulantes_merge_int = postulantes_merge.select_dtypes(include=['int','float'])
converted_int = postulantes_merge_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(postulantes_merge_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([postulantes_merge_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
optimized_postulantes_merge = postulantes_merge.copy()

optimized_postulantes_merge[converted_int.columns] = converted_int

print(mem_usage(postulantes_merge))
print(mem_usage(optimized_postulantes_merge))

In [ ]:
optimized_postulantes_merge.info()

In [19]:
optimized_postulantes_merge.to_csv('Data/optimized_postulantes_merge.csv')

In [20]:
intereses_vistas = pd.read_csv('Data/unique_postulantes_con_intereses_vistas.csv')

In [22]:
intereses_vistas.head()

,idpostulante,nombre_zona_interes,nivel_laboral_interes,tipo_de_trabajo_interes,nombre_area_interes
0,YjVJQ6Z,-7,-4,-1,-175
1,BmVpYoR,NaN,NaN,NaN,NaN
2,wVkBzZd,-7,-2,-1,-120
3,OqmP9pv,-7,-4-2,-1,-175
4,DrpbXDP,NaN,NaN,NaN,NaN
